In [1]:
!mkdir /local/shared/pinsky_lab/sequencing/hiseq_2017_06_08_SEQ21/

In [ ]:
wget -r https://htseq.princeton.edu/tmp/f2Gkd7il3EOrS9K2I/ -P /local/shared/pinsky_lab/sequencing/hiseq_2017_06_08_SEQ21/

In [ ]:
mkdir $(awk '{print $1}' /local/home/sarahgw/WGS/Sequences/Pool4/Pool4_Barcodes.txt)

Move into directories

In [ ]:
for dir in $(ls); do cd $dir; cp /local/shared/pinsky_lab/sequencing/hiseq_2017_06_08_SEQ21/"$dir"_Demultiplexed-fastq-fileset-sample-Pool-4-from-flowcell-HKMVTBCXY-on-2017-06-07_Read_1_passed_filter.fastq.gz /local/shared/pinsky_lab/sequencing/hiseq_2017_06_08_SEQ21/"$dir"_Demultiplexed-fastq-fileset-sample-Pool-4-from-flowcell-HKMVTBCXY-on-2017-06-07_Read_4_passed_filter.fastq.gz . ; cd ..; done

Run bioinfo pipeline from directory

In [ ]:
~/Notebooks/WNS_WGS/Bioinfo_pipeline.sh 

Started 06/12/17 at 9:49 AM Finished o/n 06/14/17

Accidentally just added metrics onto Pool5 metrics, so last 18 are from pool4

In [1]:
!tail -n 18 /local/home/sarahgw/WGS/Sequences/Metrics/Pool5_reads

11656101
16824534
17286857
14360827
15389811
17409127
19307485
12869482
6530028
14290835
9908658
13050385
8779282
315551
1104357
18339095
21797829
34866149


Move zip delete

In [ ]:
    rm */*passed_filter.fastq
    rm */*FU1.fastq
    rm */*FU2.fastq
    rm */*trim_p.fastq
    rm */*trim_up.fastq
    
    find . -name "*.tagged.fastq" -exec mv -i {} ../Pool4Contaminants/ \;
    for file in $(ls); do tar -zcvf "$file".tar.gz "$file" ; done
    rm *.fastq
    mkdir /Scratch/sarahgw/Pool4Contaminants/
    mv *.gz /Scratch/sarahgw/Pool4Contaminants/

Mapping both paired and unpaired reads

In [ ]:
for dir in $(ls); do cd $dir; /local/home/sarahgw/anaconda2/bin/bowtie2 --very-sensitive-local -p 12 -x /local/home/sarahgw/references/Myotis_Lucifugus2.0/MyoLuc_toplevel_index -1 $(pwd)/"$dir"_FU1_trim_p.tagged_pfilter.fastq -2 $(pwd)/"$dir"_FU2_trim_p.tagged_pfilter.fastq -U $(pwd)/"$dir"_FU1_trim_up.tagged_filter.fastq,$(pwd)/"$dir"_FU2_trim_up.tagged_filter.fastq -S $(pwd)/"$dir"_pairedplus_eg2.sam; cd ..; done

In [ ]:
for dir in $(ls); do cd $dir; java -jar /local/home/sarahgw/picard.jar SortSam \
      I=$(pwd)/"$dir"_pairedplus_eg2.sam \
      O=$(pwd)/"$dir"_pairedplus_eg2_sorted.bam \
      SORT_ORDER=coordinate; cd ..; done

In [ ]:
for dir in $(ls); do cd $dir; java -jar /local/home/sarahgw/picard.jar MarkDuplicates \
      I=$(pwd)/"$dir"_pairedplus_eg2_sorted.bam \
      O=$(pwd)/"$dir"_pairedplus_marked_dupes.bam \
      M=$(pwd)/"$dir"_pairedplus_marked_dup_metrics.txt \
      REMOVE_DUPLICATES=true \
      ASSUME_SORT_ORDER=coordinate; cd ..; done 

Calculate coverage

In [ ]:
for dir in $(ls); do cd $dir; samtools view -b $(pwd)/"$dir"_pairedplus_marked_dupes.bam | genomeCoverageBed -ibam stdin -g /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myoluc_genomeFile.txt>$(pwd)/"$dir"_Coverage.txt; cd ..; done

In [ ]:
for dir in $(ls); do cd $dir;  sed "s/$/\t"$dir"/" "$dir"_Coverage.txt > "$dir"_Coverage_name.txt ; cd ..; done

In [ ]:
cat ./*/*_Coverage_name.txt >> ./Coverage_Pool4.txt